# merge详解
merge本来准备放在上一章的增那一节讲的，不过其算是关系数据库用得很多的一个操作，变化也较多，所以单独开一篇细讲这个函数。 

In [1]:
__auther__ = 'zhenhang.sun@gmail.com'

In [2]:
pwd

'D:\\github\\pandas-tutorial'

In [3]:
import numpy as np
import pandas as pd

# 1. 函数说明

##### `pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False)`
concat函数只能根据索引对齐，而如果想在任意**列**上对齐合并，则需要merge函数，其在sql应用很多。
- left, right： 两个要对齐合并的DataFrame；
- how： 先做笛卡尔积操作，然后按照要求，保留需要的，缺失的数据填充NaN；
   - left: 以左DataFrame为基准，即左侧DataFrame的数据全部保留（不代表完全一致、可能会存在复制），保持原序;
   - right: 以右DataFrame为基准，保持原序;
   - inner: 交，保留左右DataFrame在on上完全一致的行，保持左DataFrame顺序;
   - outer: 并，按照字典顺序重新排序;
- on：对应列名或者行索引的名字，如果要在DataFrame相同的列索引做对齐，用这个参数；
- left_on, right_on, left_index, right_index：
   - on对应列名或者行索引的名字（所以行索引一般要跟列一样看待，有自己的名字），用这俩参数；
   - index对应要使用的index,不建议使用，会搞晕。
- sort: True or False，是否按字典序重新排序。

In [4]:
df1 = pd.DataFrame([[1,2],[3,4]], index = ['a','b'],columns = ['A','B'])
df2 = pd.DataFrame([[1,3],[4,8]], index = ['b','d'],columns = ['B','C'])
df1

,A,B
a,1,2
b,3,4


In [5]:
df2

,B,C
b,1,3
d,4,8


**如果单纯的按照index对齐，不如用concat方法。**

In [6]:
pd.merge(left=df1, right= df2, how='inner' ,left_index=True, right_index=True)

,A,B_x,B_y,C
b,3,4,1,3


In [7]:
# 小区别是concat对重复列没有重命名，但是重名的情况不多，而且重名了说明之前设计就不大合理。
pd.concat([df1, df2], join='inner', axis =1)  

,A,B,B,C
b,3,4,1,3


# 2. on 用法
设置 how='inner'

In [8]:
#对于'B'列：df1的'b'行、df2的'd'行，是相同的，其他都不同。 
pd.merge(left=df1, right=df2, how='inner', on=['B']) 

,A,B,C
0,3,4,8


In [9]:
# df1的'A'列'b'行，df2的'C'列'd'行是相同的，其他都不同。
# 其他列如果同名会进行重命名。
pd.merge(left=df1, right=df2, how='inner',left_on=['A'] ,right_on=['C'])

,A,B_x,B_y,C
0,3,4,1,3


# 3. how用法

In [10]:
# 保持左侧DataFrame不变，用右侧来跟它对齐，对不上的填NaN。
pd.merge(left=df1, right=df2, how='left', on=['B'] )

,A,B,C
0,1,2,NaN
1,3,4,8.0


In [11]:
# 保持右侧DataFrame不变，用右侧来跟它对齐，对不上的填NaN。
pd.merge(left=df1, right=df2, how='right', on=['B'] )

,A,B,C
0,3.0,4,8
1,NaN,1,3


**对齐的列存在重复值，重复的也没关系，操作逻辑是一致的，完全可以假想不存在重复。**

In [12]:
df1.loc['a','B'] = 4  #改成重复
df1

,A,B
a,1,4
b,3,4


In [13]:
### 保持右侧的列都在，如果左侧对齐的列存在重复值，那么对齐上后也存在重复。
pd.merge(left=df1, right=df2, how='right', on=['B'] )

,A,B,C
0,1.0,4,8
1,3.0,4,8
2,NaN,1,3
